<a href="https://colab.research.google.com/github/vivekkishore/machinelearning/blob/main/querygpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###import library and key

In [ ]:
!pip install python-dotenv
!pip install openai
!pip install --upgrade langchain
!pip install tiktoken
!pip install chromadb
import warnings
warnings.filterwarnings('ignore')
import langchain
langchain.debug=False
import os, openai
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai.api_key=os.environ['API_KEY']


###Load and split

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter

#Load
loader=TextLoader('/content/sample1.txt')
docs=loader.load()

#Split
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10,separators=["\n\n", "\n", " ", ""])
texts=text_splitter.split_documents(docs)



###LLM embedding and storing in VectorDB

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(openai_api_key=openai.api_key)


from langchain.vectorstores import Chroma
persist_directory = '/content/chroma/'

# to remove old database files
# !rm -rf ./content/chroma/

db=Chroma.from_documents(documents=texts, embedding=embedding, persist_directory=persist_directory)


###retrieval from VectorDB/ Through LLM

In [ ]:
# #fetching relevant data directly from vector db using similarilty search
# Query='Is entity id required for all kinds of applications?'
# # result=db.similarity_search(Query, k=2)
# result1=db.max_marginal_relevance_search(Query,fetch_k=4,k=3)

from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model_name='gpt-3.5-turbo',temperature=0,openai_api_key= openai.api_key)

# Build prompt
from langchain.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Run chain
from langchain.chains import RetrievalQA

def run_model(question):
  qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type='mmr',search_kwargs={'k': 3, 'fetch_k':4 }),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, verbose =True)
  result=qa_chain({"query":question})
  return result


In [ ]:
run_model("what is step 3 in SAML integration process?")

###Memory store inclusion in the flow

In [ ]:
# including memory to store past conversation
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
from langchain.chains import ConversationalRetrievalChain

def run_memory_chain(question):
  qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=db.as_retriever(search_type='similarity',search_kwargs={'k': 3, 'fetch_k':4 }),
    chain_type="stuff",
    memory=memory,
    verbose=True)

  result =qa({"question":question})
  return result



In [ ]:
run_memory_chain("what is step 3 in SAML integration process?")


# question = "why is he like this?"
# result = qa({"question": question})
# result

# question = "shall we all be like him?"
# result = qa({"question": question})
# result

# memory.clear()



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Basic SAML Configuration settings:

Step 3. Manage the SAML signing certificate
Azure AD uses a certificate to sign the SAML tokens it sends to the application. ESO and the Azure Migration team can help you to view or download the active certificate. If there is a need to update, create, or import a certificate please fill out relevant RITMs with the ESO team.

For gallery applications, details about the certificate format are available in the application’s SAML documentation (see the application-specific tutorials).

Step 2. Edit Basic SAML Configurations Create a RITM with the ESO Team
Fill out relevant RITMs with the ESO team to configure the Basic SAML configurations for single sign-on. Follow the

{'question': 'what is step 3 in SAML integration process?',
 'chat_history': [HumanMessage(content='what is step 3 in SAML integration process?', additional_kwargs={}, example=False),
  AIMessage(content='Step 3 in the SAML integration process is to manage the SAML signing certificate. This involves using a certificate to sign the SAML tokens that Azure AD sends to the application. You can view or download the active certificate with the help of the ESO team. If there is a need to update, create, or import a certificate, you should fill out relevant RITMs with the ESO team.', additional_kwargs={}, example=False)],
 'answer': 'Step 3 in the SAML integration process is to manage the SAML signing certificate. This involves using a certificate to sign the SAML tokens that Azure AD sends to the application. You can view or download the active certificate with the help of the ESO team. If there is a need to update, create, or import a certificate, you should fill out relevant RITMs with the 

In [ ]:
# memory.clear()
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history')

## Working with Azure OpenAI Key

###Simple API call

In [ ]:
# !pip install python-dotenv
# !pip install openai
import json
import openai
import os
from openai import AzureOpenAI
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

client = AzureOpenAI(
    azure_endpoint=os.environ.get("ENDPOINT"),
    api_key=os.environ.get("API_KEY"),
    api_version="2024-05-01-preview",
)

completion = client.chat.completions.create(
    model=os.environ.get("Deployment_Name"),
    messages= [
    {
      "role": "system",
      "content": "You are funny bot answering questions with joke"
    },
    {
      "role": "user",
      "content": "What are the differences between Azure Machine Learning and Azure AI services?"
    }],
    max_tokens=500,
    temperature=0.5,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False

)

print(completion.choices[0].message.content)


Why did the Azure Machine Learning and Azure AI services go to different parties? 

Because one wanted to learn the algorithms, while the other just wanted to have a good time with artificial intelligence!


###Using Langchain for simple API call

In [ ]:
!pip install -U langchain-openai
!pip install python-dotenv

from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv, find_dotenv

# # Load environment variables from the .env file
load_dotenv(find_dotenv())

# # Retrieve Azure OpenAI specific configuration from environment variables
OPENAI_API_KEY = os.getenv("API_KEY")
OPENAI_API_BASE = os.getenv("ENDPOINT")
OPENAI_API_VERSION = "2024-05-01-preview"
Deployment_Name=os.getenv("Deployment_Name")


# Initialize an instance of AzureChatOpenAI using the specified settings
chat_llm = AzureChatOpenAI(
    api_version=OPENAI_API_VERSION,
    openai_api_key=OPENAI_API_KEY,
    azure_endpoint=OPENAI_API_BASE,
    openai_api_type="azure",
    azure_deployment=Deployment_Name
)

messages = [
    {"role": "system", "content": "You are funny bot answering questions with joke"},
    {"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}
]
messages1= "Who are you?"

result=chat_llm.invoke(messages)
result
print(result.content)

Why did the AI go to the gym? 

Because it wanted some byte-sized exercise! 

But in all seriousness, Azure Machine Learning is a platform that allows you to build, deploy, and manage machine learning models. It provides tools and services for data scientists to develop and experiment with their models.

On the other hand, Azure AI services are a collection of pre-built AI capabilities that can be easily integrated into your applications. These services include computer vision, natural language processing, speech recognition, and more. They are designed to provide ready-to-use AI functionality without the need for extensive machine learning expertise.

So, in a nutshell, Azure Machine Learning is more focused on building and training models, while Azure AI services offer out-of-the-box AI capabilities for various tasks.


In [ ]:
print("AzureOpenAI ChatLLM Response: ", chat_llm.predict("you are funny bot, tell me about big bang theory"))

AzureOpenAI ChatLLM Response:  Thank you! I'm glad you find me funny. As for "The Big Bang Theory," it's a popular American sitcom that aired from 2007 to 2019. The show revolves around a group of socially awkward but highly intelligent scientists, Leonard, Sheldon, Howard, and Raj, along with their neighbor Penny. The series humorously explores their personal lives, friendships, romantic relationships, and their adventures in the world of science. It's known for its witty dialogue, geeky references, and iconic catchphrases. If you're a fan of nerdy humor and pop culture references, you might enjoy watching it!


###Using Langchain for vector Indexing - Still to verify

In [ ]:
# prompt: write code to query a dataset using Azure openai key and langchain

# ###Using Langchain for vector Indexing
# Load environment variables from the .env file
load_dotenv(find_dotenv())

# Retrieve Azure OpenAI specific configuration from environment variables
OPENAI_API_KEY = os.getenv("API_KEY")
OPENAI_API_BASE = os.getenv("ENDPOINT")
OPENAI_API_VERSION = "2024-05-01-preview"
Deployment_Name=os.getenv("Deployment_Name")

# Initialize Azure OpenAI embeddings
embeddings = OpenAIEmbeddings(
    deployment=Deployment_Name,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE,
    openai_api_version=OPENAI_API_VERSION,
    chunk_size=1
)

# Load documents
loader=TextLoader('/content/sample1.txt')
docs=loader.load()

# Split documents
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10,separators=["\n\n", "\n", " ", ""])
texts=text_splitter.split_documents(docs)

# Create Chroma database
persist_directory = '/content/chroma/'
db=Chroma.from_documents(documents=texts, embedding=embeddings, persist_directory=persist_directory)

# Initialize AzureChatOpenAI LLM
chat_llm = AzureChatOpenAI(
    api_version=OPENAI_API_VERSION,
    openai_api_key=OPENAI_API_KEY,
    azure_endpoint=OPENAI_API_BASE,
    openai_api_type="azure",
    azure_deployment=Deployment_Name
)

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template)

# Run chain
def run_model(question):
  qa_chain = RetrievalQA.from_chain_type(chat_llm,
                                       retriever=db.as_retriever(search_type='mmr',search_kwargs={'k': 3, 'fetch_k':4 }),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}, verbose =True)
  result=qa_chain({"query":question})
  return result

# Query the dataset
run_model("what is step 3 in SAML integration process?")


<class 'langchain_core.messages.ai.AIMessage'>


In [ ]:
llm = chat_llm

llm_prompt = PromptTemplate(
    input_variables=["human_prompt"],
    template="The following is a conversation with an AI assistant. The assistant is helpful.\n\nAI: I am an AI created by OpenAI. How can I help you today?\nHuman: {human_prompt}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=llm_prompt)

prompt="Wht is your name?"
return chain.run(prompt) # prompt is human input from request body

NameError: name 'chat_llm' is not defined

###Langchain CSV Agent

In [ ]:
!pip install langchain langchain_experimental langchain_openai tabulate
!pip install python-dotenv

In [3]:
import os
from dotenv import dotenv_values
config=dotenv_values("val.env")
from langchain_openai import AzureOpenAI, AzureChatOpenAI

llm=AzureOpenAI(
    api_version=config['Model_Version'],
    openai_api_key=config["API_KEY"],
    azure_endpoint=config["ENDPOINT"],
    openai_api_type=config["GPT_Model_Name"],
    azure_deployment=config["Deployment_Name"]

)

chat_llm = AzureChatOpenAI(
   api_version=config['Model_Version'],
    openai_api_key=config["API_KEY"],
    azure_endpoint=config["ENDPOINT"],
    openai_api_type=config["GPT_Model_Name"],
    azure_deployment=config["Deployment_Name"]
)

In [4]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent
agent = create_csv_agent(
    llm=chat_llm,
    path="country.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True
)

In [6]:
res=agent.run("what details you have?")
res



> Entering new AgentExecutor chain...
Thought: I need to check the details of the dataframe to see what information is available.
Action: Use the `df.info()` function to get the details of the dataframe.
Action Input: `df.info()`Use the `df.info()` function to get the details of the dataframe. is not a valid tool, try one of [python_repl_ast].

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I need to use the `python_repl_ast` tool to execute the `df.info()` command.`